In [ ]:
import pandas as pd

# Load the dataset
file_path = 'churn_data.csv'
df = pd.read_csv(file_path)

# Basic Analysis

## What is the overall churn rate?

## How does the churn rate differ between make and female customers?

## What are the most common reasons customers cite for churning?

## What is the average tenure of customers who churned?

## What is the average tenure of customers who churned?